# Week 3 - Neighborhoods of São Paulo


## Introduction/Business Problem 

The City of São Paulo, is the most populous city in Brazil. It is the financial capital of Brazil. Many travelers do not know the city very well and had trouble finding the best places to stay, when visiting the city. This projects aims to help find hotels, restaurants that are closed to the main place that the visitor was interested in the first place and will try to help find the best hotel to meet his needs.

# Data Section

For this project will use the data available in the site "http://dados.prefeitura.sp.gov.br/", who provides a rich source of data from the site in csv files and other formats. Due to the good amount of data available we can make a powerfull code to address the problem. I will devide the city in the main regions and then sort the busiest places in the city and try to locate hotels nearby this areas and then offer the user to set a location that he was already interested to find the best match for hes needs.

In [1]:
import numpy as np # library to handle data in a vectorized manner
! pip install geopy
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [2]:

url1 = 'https://raw.githubusercontent.com/marcellocaron/geojs-35-mun/master/municipio-sp.geojson'

url3 = 'https://raw.githubusercontent.com/marcellocaron/geojs-35-mun/master/municipio-sp.geojson'

url2 = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/35/microrregioes'


In [3]:
CLIENT_ID = 'W4S5JOQEZJSSGHS5ERZAUZPVLCATG3K3K1VX3RMXTDWAR15N' # your Foursquare ID
CLIENT_SECRET = 'RT2U4CRDWZEZOKVMILVPHOB1WBSWEEIPOZ5HE4NNMOFWSRY1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W4S5JOQEZJSSGHS5ERZAUZPVLCATG3K3K1VX3RMXTDWAR15N
CLIENT_SECRET:RT2U4CRDWZEZOKVMILVPHOB1WBSWEEIPOZ5HE4NNMOFWSRY1


In [4]:
results = requests.get(url1).json()


In [5]:
address = 'Metropolitana de São Paulo, São Paulo, SP'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-23.71865045 -46.7010388456856


In [6]:
neighborhoods_data = results['features']

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-46.648614, -23.567189],
     [-46.653394, -23.563265],
     [-46.65861, -23.558781],
     [-46.655482, -23.554765],
     [-46.6554, -23.554698],
     [-46.650411, -23.552023],
     [-46.649904, -23.551756],
     [-46.647387, -23.550947],
     [-46.646899, -23.551161],
     [-46.646683, -23.550765],
     [-46.646456, -23.550576],
     [-46.646039, -23.550397],
     [-46.645077, -23.552982],
     [-46.644912, -23.553369],
     [-46.644738, -23.553642],
     [-46.64446, -23.553888],
     [-46.644176, -23.554064],
     [-46.642248, -23.554803],
     [-46.64161, -23.554973],
     [-46.639819, -23.555266],
     [-46.639603, -23.555888],
     [-46.639253, -23.55536],
     [-46.637277, -23.555709],
     [-46.638272, -23.558304],
     [-46.638481, -23.558662],
     [-46.639175, -23.559671],
     [-46.639494, -23.560322],
     [-46.639639, -23.560699],
     [-46.6397, -23.561077],
     [-46.639896, -23.562973],
     

In [8]:
# define the dataframe columns
column_names = ['Bairro', 'Coordenadas'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,Bairro,Coordenadas


In [9]:
for data in neighborhoods_data:
    
    borough = neighborhood_name = data['properties']['id'] 
    neighborhood_name = data['properties']['nome']
    
    ident = data['properties']['cartodb_id'] 
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[0]

    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'ID': ident,
                                          'Bairro': neighborhood_name,
                                          'Coordenadas': neighborhood_lat,
                                           },ignore_index=True)

In [10]:
neighborhoods.head()

,Bairro,Coordenadas,Borough,ID
0,Bela Vista,"[[[-46.648614, -23.567189], [-46.653394, -23.5...",22.0,34.0
1,Liberdade,"[[[-46.640735, -23.572888], [-46.640488, -23.5...",24.0,38.0
2,Pari,"[[[-46.624928, -23.519002], [-46.623455, -23.5...",259.0,42.0
3,Mooca,"[[[-46.603601, -23.566726], [-46.604565, -23.5...",30.0,21.0
4,Vila Andrade,"[[[-46.727635, -23.61106], [-46.721546, -23.61...",3.0,8.0


In [11]:
 
sf_geo = results
world_map = folium.Map()

world_map = folium.Map(location=[latitude, longitude], zoom_start=10.4)

world_map.choropleth(
    geo_data= sf_geo,
    data= neighborhoods,
    columns=['Bairro','ID'],
    key_on='feature.properties.nome',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='SP'
)


# display map
world_map


In [13]:
# Here you choose the neighborhood you want stay
# For this example I choose "Itaim Bibi"

want_stay_data = neighborhoods[neighborhoods['Bairro'] == 'Itaim Bibi'].reset_index(drop=True)
want_stay_data.head()

,Bairro,Coordenadas,Borough,ID
0,Itaim Bibi,"[[[-46.70093, -23.61956], [-46.700576, -23.618...",9.0,29.0


In [28]:
# You can put the neighborhood that you want to stay and see an overwiew of the area

address = 'Itaim Bibi, SP'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))


# create map using latitude and longitude values

map_want_stay = folium.Map(location=[latitude, longitude], zoom_start=14.2)

map_want_stay.choropleth(
    geo_data= sf_geo,
    data= want_stay_data,
    columns=['Bairro','ID'],
    key_on='feature.properties.nome',
    fill_color='YlOrRd', 
    fill_opacity=0.2, 
    line_opacity=0.2,
    legend_name='Your place to stay'
)

map_want_stay

The geograpical coordinate are -23.601511700000003, -46.68520798778679.


In [32]:
# Here you have to insert what you want to search in the neighborhood. Example Hotels, Resstaurants...

search_query = 'Hotel'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

search_sp = requests.get(url).json()


Hotel .... OK!


In [33]:
# assign relevant part of JSON to venues
venues = search_sp['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4f0d65a8e4b0b30606683637,Mix Park - Hotel Transamérica Prime,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1587243428,False,"Rua Gomes de Carvalho, 1005",Rua Alvorada,-23.598232,-46.682840,"[{'label': 'display', 'lat': -23.5982316771113...",437,BR,São Paulo,SP,Brasil,"[Rua Gomes de Carvalho, 1005 (Rua Alvorada), S...",NaN
1,5cafad80c53093002c45d1fa,Hotel The World,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587243428,False,NaN,NaN,-23.598429,-46.682639,"[{'label': 'display', 'lat': -23.598429, 'lng'...",431,BR,São Paulo,SP,Brasil,"[São Paulo, SP, Brasil]",NaN
2,4bcf2da6a8b3a5933aab615f,Quality Suites Long Stay Vila Olímpia,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587243428,False,"R. Alvorada, 1009",NaN,-23.600016,-46.682934,"[{'label': 'display', 'lat': -23.6000160704025...",285,BR,São Paulo,SP,Brasil,"[R. Alvorada, 1009, São Paulo, SP, 04550-004, ...",04550-004
3,54d81c7f498e8b6a062b9cd0,Travel Inn The World,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587243428,False,"R. Gomes de Carvalho, 1005",R. Alvorada,-23.598042,-46.683326,"[{'label': 'display', 'lat': -23.5980415437969...",431,BR,São Paulo,SP,Brasil,"[R. Gomes de Carvalho, 1005 (R. Alvorada), São...",04547-004
4,525740c1498e0073cc5bb693,Atlantica Hotels International,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",v-1587243428,False,NaN,NaN,-23.600835,-46.682057,"[{'label': 'display', 'lat': -23.6008350372208...",330,BR,NaN,NaN,Brasil,[Brasil],NaN


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Mix Park - Hotel Transamérica Prime,Parking,"Rua Gomes de Carvalho, 1005",Rua Alvorada,-23.598232,-46.682840,"[{'label': 'display', 'lat': -23.5982316771113...",437,BR,São Paulo,SP,Brasil,"[Rua Gomes de Carvalho, 1005 (Rua Alvorada), S...",NaN,4f0d65a8e4b0b30606683637
1,Hotel The World,Hotel,NaN,NaN,-23.598429,-46.682639,"[{'label': 'display', 'lat': -23.598429, 'lng'...",431,BR,São Paulo,SP,Brasil,"[São Paulo, SP, Brasil]",NaN,5cafad80c53093002c45d1fa
2,Quality Suites Long Stay Vila Olímpia,Hotel,"R. Alvorada, 1009",NaN,-23.600016,-46.682934,"[{'label': 'display', 'lat': -23.6000160704025...",285,BR,São Paulo,SP,Brasil,"[R. Alvorada, 1009, São Paulo, SP, 04550-004, ...",04550-004,4bcf2da6a8b3a5933aab615f
3,Travel Inn The World,Hotel,"R. Gomes de Carvalho, 1005",R. Alvorada,-23.598042,-46.683326,"[{'label': 'display', 'lat': -23.5980415437969...",431,BR,São Paulo,SP,Brasil,"[R. Gomes de Carvalho, 1005 (R. Alvorada), São...",04547-004,54d81c7f498e8b6a062b9cd0
4,Atlantica Hotels International,Building,NaN,NaN,-23.600835,-46.682057,"[{'label': 'display', 'lat': -23.6008350372208...",330,BR,NaN,NaN,Brasil,[Brasil],NaN,525740c1498e0073cc5bb693
5,Ristorante Victorino,Brazilian Restaurant,"R. Gomes de Carvalho, 1005",NaN,-23.598563,-46.683227,"[{'label': 'display', 'lat': -23.5985634173148...",385,BR,São Paulo,SP,Brasil,"[R. Gomes de Carvalho, 1005, São Paulo, SP, Br...",NaN,4b82a04ef964a52051db30e3
6,Copa Hotelera Posadas,Café,NaN,NaN,-23.597146,-46.684599,"[{'label': 'display', 'lat': -23.5971457565654...",489,BR,NaN,NaN,Brasil,[Brasil],NaN,4f8c8584e4b07efba0b2c8bf
7,serhs hotels brasil,General Travel,Av. Doutor Cardoso De Melo 1470,NaN,-23.596931,-46.687156,"[{'label': 'display', 'lat': -23.5969310026281...",547,BR,NaN,NaN,Brasil,"[Av. Doutor Cardoso De Melo 1470, Brasil]",NaN,534eb8c511d219a6d1996bc0


In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [44]:
# Now we want to know what venues are trending in the area

if len(search_sp['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = search_sp['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
    

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [51]:
if len(search_sp['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=20,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        

In [52]:
venues_map

# Results

This project can be use to search hotels, restaurants and other locations in the city of São Paulo, Brazil. It will help you make a better decision based on the locations you want to go in the first place.

This project has shown me a practical application to resolve a real situation that has impacting personal and financial impact using Data Science tools. The mapping with Folium is a very powerful technique to consolidate information and make the analysis and decision thoroughly and with confidence. I would recommend for use in similar situations.